In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from numpy.linalg import inv
import numpy.linalg as LA

In [2]:
class LogisticRegression(object):
    def __init__(self,lr,n_iters,weight,x,y):
        self.lr = lr
        self.n_iters = n_iters
        self.weight = weight
        self.x = x
        self.y = y
        
    def Sigmoid(self,z):
        sign = 1 / (1+np.exp(-z))
        return sign
    
    def Cost(self):
        yhat= self.x@self.weight
        cost = ((-self.y.T) @ np.log2(self.Sigmoid(yhat))) - ((1 - self.y).T @ np.log2(1 - self.Sigmoid(yhat)))
        #print(yhat.shape)
        return cost
    
    def Gradient(self):
        yhat= self.x@self.weight
        return self.x.T @ ((self.y) - self.Sigmoid(yhat))
    
    def fit(self):
        loss = np.zeros((self.n_iters))
        for i in range(self.n_iters):
            self.weight = self.weight + (self.lr * self.Gradient())
            
            loss[i] = self.Cost()
            
        return self.weight, loss
    
    
#     def pred_prob(self,z):
    
    
#         return self.Sigmoid(np.dot(z,self.weight))
        
#     def predict(self,prediction):
    
#         liste = []

#         for elt in prediction:
#             if elt<0.5:
#                 liste.append(0)
#             else:
#                 liste.append(1)
#         liste = np.array(liste)
#         liste = liste.reshape(-1,1)
#         return liste
    
    def hessian(self):
        yhat = self.x@self.weight
        omega = self.Sigmoid(yhat) @ (1 - self.Sigmoid(yhat)).T
        return ((-x.T) @ omega)@x -2.44641553e+05, LA.eigvals(((-x.T) @ omega)@x)

    def Newton(self):
        for i in range(self.n_iters):
            self.weight = self.weight - inv(self.hessian()[0])@self.Gradient()
        return self.weight
    
    def Good_pred(self,z):
        
        yhat=z@self.weight
        
        return np.where(self.Sigmoid(yhat)>0.5,1,0)
    
    
def accuracy(prediction,y):
    acc = y==prediction
    return np.sum(acc)/len(y)
        
        

In [3]:
def Sigmoid(x):
    sign = 1 / (1+np.exp(-x))
    return sign

In [4]:
def Good_pred(z, weight):
        
        yhat=z@weight
        
        return np.where(Sigmoid(yhat)>0.5,1,0)
    

In [5]:
from sklearn import datasets
iris=datasets.load_iris()   
x=iris.data[:,:4]
Y=(iris.target!=0)*1
Y=Y.reshape(-1,1)
theta = np.zeros((x.shape[1],1))
        

In [6]:
result=LogisticRegression(0.01,10000,theta,x,Y).Newton()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [7]:
result

array([[-5.71457045e+12],
       [ 2.03883375e+12],
       [ 1.12603750e+13],
       [-7.58463831e+12]])

In [8]:
result=LogisticRegression(0.01,10000,theta,x,Y).fit()


/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log2
  from ipykernel import kernelapp as app
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kernelapp as app


In [9]:
result[0]

array([[-1.30726813],
       [-4.10517719],
       [ 6.40807657],
       [ 3.07061467]])

In [10]:
pred1= Good_pred(x,result[0])
theta = np.zeros((x.shape[1],1))

In [11]:
theta = np.zeros((x.shape[1],1))


### K-fold cross validation

In [12]:
def k_fold(k,data):
    #x_train = portion[data[:20]]
    #data = np.random.permutation(data)
    liste = []
    size = int(len(data) / k)
    for i in range(k):
      #  import pdb
       # pdb.set_trace()
        data_new = data[i*size:(i+1)*size,:]
        liste.append(data_new)
    if len(data) % k ==0:
        return liste
    else:
        liste[0] = np.vstack((liste[0],data[(len(data)-1),:]))
        return liste
    #return liste

In [13]:
test =k_fold(5,x)

In [14]:
test1 = np.concatenate((test[0],test[1]),axis =0)

In [15]:
test1.shape

(60, 4)

In [16]:
# def cross_validation(x,y,k):
#     x = k_fold(k,x)
#     y = k_fold(k,y)
#     s = 1#iteration
#     n = 0#iteration
#     for i in range(k):
#         x_test = x[n:s,]
#         x_train = np.concatenate((x[s:][0],x[:n,][0]), axis=0)
#         n = n+1
#         s = s+1
#         result=LogisticRegression(0.01,100,theta,x_train,y_train).fit()
#         #predict=LogisticRegression(0.01,100,result[0],x,y).pred_prob(x)
#         #pred=LogisticRegression(0.01,100,result[0],x,y).predict(predict)
#         print(result)
    

In [17]:
def K_fold_cv(x,y,k):
    
    x=np.random.permutation(x)
    y=np.random.permutation(y)
    #l1=np.random.permutation(len(x))
    #l2=np.random.permutation(len(y))
    #x=x[l1]
    #y=y[l2]
    size =  int(len(x)/k)
    i = 0
    size1 = int (len(x)/k)
    theta = np.zeros((x.shape[1],1))
    acc1= []
    for j in range(k):
        x_test = x[i:size,]
        x_train = np.concatenate((x[size:,],x[:i,]),axis=0)
        y_test = y[i:size,]
        y_train = np.concatenate((y[size:,],y[:i,]),axis=0)
        weight = LogisticRegression(0.01,100,theta,x_train,y_train).fit()[0]
        
        #validation = LogisticRegression(0.01,10000,weight,x_test,y_test).Good_pred(x_test)
        #validation= Sigmoid(x_test@weight)
        validation = Good_pred(x_test, weight)
        #y_pred = LogisticRegression(0.01,10000,weight,x_train,y_train).predict(validation)
        #acc = accuracy(y_pred,y_test)
        acc = accuracy(validation,y_test)
        size += size
        i = i+size1
    
        acc1.append(acc)
        #print (acc)
    return np.mean(acc1)
        

In [18]:
K_fold_cv(x,Y,4)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log2
  from ipykernel import kernelapp as app
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kernelapp as app


0.6353083853083853

In [19]:
import statsmodels.api as sm

In [20]:
def backward_elimination(data, target, significance_level):
    feature=data.columns.tolist()
    while(len(feature)>0):
        feature_with_constant=sm.add(data[feature])
        p_value=sm.OLS(target,feature_with_constant).fit().pvalue[1:]
        max_p_value=p_value.max()
        if max_p_value>=significance_level:
            removed_feature=p_value.idxmax()
            feature.remove(removed_feature)
        else:
            break
            return feature

In [21]:
from sklearn import datasets
iris=datasets.load_iris()   
x=iris.data[:,:4]
Y=(iris.target!=0)*1
Y=Y.reshape(-1,1)
theta = np.zeros((x.shape[1],1))

In [22]:
def feature_selection(X,y,k):
    cmpt=0
    f=1
    for j in range(X.shape[1]):
        X_non_used=X[:,cmpt:f]
        X_used=np.concatenate((X[:,f:],X[:,:cmpt]),axis=1)
        #import pdb
        #pdb=pdb.set_trace()
        cmpt+=1
        f+=1
        #K_fold_cv(X_used,y,k)
        print( K_fold_cv(X_used,y,k))
        #print(X_used)

In [24]:
feature_selection(x,Y,20)

0.5837020727681257
0.6166908965441676
0.6223330597769197
0.6050397409217174


/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log2
  from ipykernel import kernelapp as app
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kernelapp as app


In [71]:
#x=pd.DataFrame(x)

In [72]:
#x.iloc[:,0:1]

### DataFrame

In [ ]:
def K_fold_cv_data(x,y,k):
    
    x=np.random.permutation(x)
    y=np.random.permutation(y)
    #l1=np.random.permutation(len(x))
    #l2=np.random.permutation(len(y))
    #x=x[l1]
    #y=y[l2]
    size =  int(len(x)/k)
    i = 0
    size1 = int (len(x)/k)
    theta = np.zeros((x.shape[1],1))
    acc1= []
    for j in range(k):
        x_test = x[i:size,]
        x_train = np.concatenate((x[size:,],x[:i,]),axis=0)
        y_test = y[i:size,]
        y_train = np.concatenate((y[size:,],y[:i,]),axis=0)
        weight = LogisticRegression(0.01,100,theta,x_train,y_train).fit()[0]
        
        #validation = LogisticRegression(0.01,10000,weight,x_test,y_test).Good_pred(x_test)
        #validation= Sigmoid(x_test@weight)
        validation = Good_pred(x_test, weight)
        #y_pred = LogisticRegression(0.01,10000,weight,x_train,y_train).predict(validation)
        #acc = accuracy(y_pred,y_test)
        acc = accuracy(validation,y_test)
        size += size
        i = i+size1
    
        acc1.append(acc)
        #print (acc)
    return np.mean(acc1)


In [1]:
def backward_elimination(data, target, significence_level=0.05):
    feature=data.columns.tolist()
    while(len(feature)>0):
        feature_with_constant=sm.add(data[feature])
        p_value=sm.OLS(target, feature_with_constant).fit().pvalue[1:]
        max_p_value=p_value.max()
        if max_p_value>significence_level:
            retracted_feature=p_value.idxmax()
            feature.remove(retracted_feature)
        else:
            break
            return feature

In [3]:
def k_fold_cv(k,x):
    split=[]
    m=len(x)
    size=int(m/k)
    for i in range(d):
        x_new=x[i*size:(i+1)*size,:]
        split.append(x_new)
        x_new=0
    if m%k==0:
        return split
    else:
        split[0]=np.vstack((split[0],x[m-1,]))
        return split

In [ ]:
def K_fold_cv(k,x,y):
    m=len(x)
    size0=int(m/k)
    size1=int(m/k)
    i=0
    for j in range(k):
        x_test=x[i:size,]
        x_train=np.concatenate((x[size:,],x[:i,]), axis=0)
        y_test=y[i:size,]
        y_train=np.concatenate((y[size:,],y[:i,]),axis=0)
        validation=